In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

engine = create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/', connect_args={'connect_timeout': 10})

scratch_sql = """SHOW DATABASES"""

scratch_data = pd.read_sql(scratch_sql, engine)
#print(scratch_data)

In [2]:
engine = create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/skynet', connect_args={'connect_timeout': 10})

scratch_sql = """SHOW TABLES"""
scratch_data = pd.read_sql(scratch_sql, engine)
#print(scratch_data)

In [3]:
scratch_sql = """select user_id, user_agent from bh_metrics"""
scratch_data = pd.read_sql(scratch_sql, engine)

In [4]:
electron_count = 0
bh_ids = []
for i in range(0, len(scratch_data.index)):
    id = scratch_data['user_id'][i]
    agent = scratch_data['user_agent'][i]
    if not agent or 'bh_electron' not in agent: continue
    if agent == 'bh_electron_microsoft': continue
    electron_type = agent.replace('bh_electron_{"client_game_id":"thunderrun","client_vendor":"','').replace('","client_build_number":1}','')
    if electron_type == 'microsoft' and id not in bh_ids:
        bh_ids.append(id)

print(len(bh_ids))

279


In [6]:
engine = create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/tr_upcache', connect_args={'connect_timeout': 10})

scratch_sql = """select sum(money_spent) from tr_upcache WHERE"""
scratch_sql += ' bh_id = "' +bh_ids[1] + '"'
for id in bh_ids[2:]:
    if not id: continue
    scratch_sql += ' OR bh_id = "' + id + '"'


scratch_data = pd.read_sql(scratch_sql, engine)


print(scratch_data)

   sum(money_spent)
0      35805.047149
